In [1]:
import json,folium
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
gpark = pd.read_csv('data/경기도도시공원정보표준데이터.csv', encoding='euc-kr')
gpark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17525 entries, 0 to 17524
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   관리번호          17525 non-null  object 
 1   공원명           17525 non-null  object 
 2   공원구분          17525 non-null  object 
 3   소재지도로명주소      3692 non-null   object 
 4   소재지지번주소       17335 non-null  object 
 5   위도            17513 non-null  float64
 6   경도            17514 non-null  float64
 7   공원면적          17525 non-null  float64
 8   공원보유시설(운동시설)  4821 non-null   object 
 9   공원보유시설(유희시설)  6200 non-null   object 
 10  공원보유시설(편익시설)  4842 non-null   object 
 11  공원보유시설(교양시설)  805 non-null    object 
 12  공원보유시설(기타시설)  2577 non-null   object 
 13  지정고시일         14451 non-null  object 
 14  관리기관명         16033 non-null  object 
 15  전화번호          16343 non-null  object 
 16  데이터기준일자       17525 non-null  object 
 17  제공기관코드        17525 non-null  object 
 18  제공기관명         17525 non-nu

In [4]:
gpark.head(1)

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
0,11170-00048,녹색소공원,소공원,NaN,서울특별시 용산구 효창동 287-6,37.539217,126.962694,1569.9,NaN,NaN,NaN,NaN,NaN,2012-10-19,서울특별시 용산구청,02-2199-7605,2022-12-20,3020000,서울특별시 용산구


In [5]:
gp = gpark[['공원명','공원면적','소재지지번주소','경도', '위도']]
gp.head()

,공원명,공원면적,소재지지번주소,경도,위도
0,녹색소공원,1569.9,서울특별시 용산구 효창동 287-6,126.962694,37.539217
1,푸름샘 소공원(소1-1),1820.0,NaN,127.233781,36.516117
2,솔숲 소공원(소1-2),1037.0,NaN,127.230040,36.508995
3,마디꽃 수변공원(수1-1),4928.0,NaN,127.257316,36.495962
4,꽃 수변공원(수2-3),3575.0,NaN,127.254482,36.473133


In [6]:
gp.dropna(subset=['소재지지번주소'], inplace=True)
gp.isna().sum() 

공원명         0
공원면적        0
소재지지번주소     0
경도         11
위도         12
dtype: int64

In [7]:
# 결측치 제거
gp.dropna(subset=['경도','위도'], inplace=True)
gp.isna().sum() 

공원명        0
공원면적       0
소재지지번주소    0
경도         0
위도         0
dtype: int64

In [8]:
type(gp.공원면적[0])

numpy.float64

In [9]:
gp.reset_index(inplace=True)
gp.head()

,index,공원명,공원면적,소재지지번주소,경도,위도
0,0,녹색소공원,1569.9,서울특별시 용산구 효창동 287-6,126.962694,37.539217
1,23,강변공원,1766.0,서울특별시 용산구 이촌동 408,126.975039,37.521338
2,24,솔밭어린이공원,2090.0,서울특별시 용산구 한강로1가 30-3,126.973668,37.537229
3,25,신창동어린이공원,257.0,서울특별시 용산구 신창동 100,126.954145,37.536698
4,26,서빙고공원,11879.0,서울특별시 용산구 용산동5가 24-2,126.971283,37.524782


In [10]:
gp.columns

Index(['index', '공원명', '공원면적', '소재지지번주소', '경도', '위도'], dtype='object')

In [11]:
del gp['index']
gp

,공원명,공원면적,소재지지번주소,경도,위도
0,녹색소공원,1569.9,서울특별시 용산구 효창동 287-6,126.962694,37.539217
1,강변공원,1766.0,서울특별시 용산구 이촌동 408,126.975039,37.521338
2,솔밭어린이공원,2090.0,서울특별시 용산구 한강로1가 30-3,126.973668,37.537229
3,신창동어린이공원,257.0,서울특별시 용산구 신창동 100,126.954145,37.536698
4,서빙고공원,11879.0,서울특별시 용산구 용산동5가 24-2,126.971283,37.524782
...,...,...,...,...,...
17309,방배,210517.2,서울특별시 서초구 방배동 686-201,126.995298,37.474225
17310,말죽거리,280822.6,서울특별시 서초구 서초동 산28,127.030469,37.480885
17311,서리풀,465938.3,서울특별시 서초구 방배동 산23-1,127.000420,37.484654
17312,방배,74847.6,서울특별시 서초구 방배동 산 43-7,126.989600,37.470535


In [12]:
gp.공원면적 = gp.공원면적.astype(int)

In [13]:
gdf = gp[gp.소재지지번주소.str.contains('경기도')]
gdf.head(80).tail(10)

,공원명,공원면적,소재지지번주소,경도,위도
406,제86호 어린이공원(한아름),1592,경기도 용인시 기흥구 상하동 631,127.136701,37.273601
407,제49호 근린공원(김혁근린공원),21920,경기도 용인시 기흥구 구갈동 569,127.127801,37.275919
408,제48호 근린공원(강남근린공원),189796,경기도 용인시 기흥구 구갈동 598,127.129559,37.272744
409,제51호 어린이공원(하늘채),1615,경기도 용인시 기흥구 구갈동 549,127.129401,37.278008
410,제52호 어린이공원(갈곡),1501,경기도 용인시 기흥구 구갈동 584,127.129607,37.271801
411,제53호 어린이공원(강남),1500,경기도 용인시 기흥구 구갈동 602,127.130383,37.269901
412,제54호 어린이공원(은하수),1500,경기도 용인시 기흥구 구갈동 605,127.134001,37.270230
413,보정제1호 어린이공원(신촌),1528,경기도 용인시 기흥구 보정동 1201,127.112514,37.321800
414,보정제2호 어린이공원(꽃메),3324,경기도 용인시 기흥구 보정동 1211,127.116229,37.320785
415,보정제3호 어린이공원(독정),3363,경기도 용인시 기흥구 보정동 1235,127.120974,37.319442


In [14]:
gdf['자치구'] = gdf.소재지지번주소.apply(lambda x: ''.join(x.split()[1:3]) if x.find('구 ') >= 0 else x.split()[1])
gdf

,공원명,공원면적,소재지지번주소,경도,위도,자치구
190,꿈나무공원,2526,경기도 이천시 중리동 15,127.438819,37.278561,이천시
191,남천공원,3830,경기도 이천시 중리동 210,127.444436,37.278010,이천시
192,향교공원,1757,경기도 이천시 창전동 408,127.437589,37.284902,이천시
193,개나리공원,1650,경기도 이천시 창전동 414-13,127.439477,37.283860,이천시
194,달맞이공원,1650,경기도 이천시 창전동 423,127.443215,37.285854,이천시
...,...,...,...,...,...,...
16824,제90호 어린이공원(풀향기),1500,경기도 용인시 기흥구 중동 1083,127.144701,37.277940,용인시기흥구
17057,청석공원,193530,경기도 광주시 경안동 242,127.264843,37.411992,광주시
17058,경안근린공원,83095,경기도 광주시 경안동 168-1,127.251285,37.409360,광주시
17059,삼리어린이공원,1493,경기도 광주시 곤지암읍 삼리 131-1,127.326794,37.356423,광주시


In [15]:
gdf1 = gdf.groupby(['자치구'])[['공원면적']].agg(['sum'])
gdf1.head()

,공원면적
,sum
자치구,
가평군,511673
고양시덕양구,3287283
고양시일산동구,2722980
고양시일산서구,922486
과천시,315666


In [16]:
gdf1.columns = ['공원면적']
gdf1.head()

,공원면적
자치구,
가평군,511673
고양시덕양구,3287283
고양시일산동구,2722980
고양시일산서구,922486
과천시,315666


In [17]:
gdf1.index

Index(['가평군', '고양시덕양구', '고양시일산동구', '고양시일산서구', '과천시', '광명시', '광주시', '구리시',
       '군포시', '김포시', '남양주시', '동두천시', '부천시', '성남시분당구', '성남시수정구', '성남시중원구',
       '수원시권선구', '수원시영통구', '수원시장안구', '수원시팔달구', '시흥시', '안산시단원구', '안산시상록구',
       '안성시', '안양시동안구', '안양시만안구', '양주시', '양평군', '여주시', '연천군', '오산시', '용인시기흥구',
       '용인시수지구', '용인시처인구', '의왕시', '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시',
       '화성시'],
      dtype='object', name='자치구')

In [18]:
file = 'data/경기도_geo_simple.json'
geo_data = json.load(open(file, encoding='utf-8'))

In [19]:
import numpy as np
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']: # 경도,위도 표시
            geo = np.array(coord)
            gu_dict[gu['properties']['name']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]  # 경도,위도 표시
    return gu_dict

In [20]:
gu_dict = get_text_location(geo_data) # /경기도_geo_simple
gu_dict

{'양평군': [37.524478319941124, 127.57566999921767],
 '가평군': [37.80580217057866, 127.46168954149552],
 '연천군': [38.089173518920326, 126.98309880050942],
 '여주시': [37.30448255424536, 127.59166773209084],
 '포천시': [37.96989171146047, 127.23835163385141],
 '양주시': [37.79323667275805, 126.99994734085243],
 '광주시': [37.398346843618, 127.2921094056205],
 '화성시': [37.1612546033386, 126.88516922146444],
 '김포시': [37.667693477291564, 126.63558015047269],
 '안성시': [37.037984261825656, 127.31341594503061],
 '이천시': [37.198345154907365, 127.48867855904936],
 '파주시': [37.86614979102598, 126.8151997375004],
 '용인시수지구': [37.33143354615715, 127.07814257535223],
 '용인시기흥구': [37.26775157395476, 127.114890702286],
 '용인시처인구': [37.19555064723713, 127.26846597201227],
 '하남시': [37.51884197502213, 127.20638379320309],
 '의왕시': [37.35860200565407, 126.99342804765003],
 '군포시': [37.34247803452407, 126.92866515577917],
 '시흥시': [37.38683244726705, 126.79291699598996],
 '오산시': [37.15601650974868, 127.06212082657765],
 '남양주시': [37.

In [21]:
map = folium.Map([37.55, 126.98], zoom_start=8, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data, # 맵데이터
    data=gdf1.공원면적, # 단계구분도로 보여줄 데이터
    columns=[gdf1.index, gdf1.공원면적], # dataframe에서 추출할 컬럼(항목)
    fill_color='YlGnBu',   # Colormap
    key_on='properties.name' # 지도에서 join할 항목
).add_to(map)
for gu_name in gdf1.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
                              html=f'<div style="font-size: 10pt">{gu_name}</div>')
    ).add_to(map)
map
title_html = '<h3 align="center" style="font-size:20px">경기도 시군별 공원면적</h3>' # 지도 제목달기
map.get_root().html.add_child(folium.Element(title_html))
map

In [22]:
pop = pd.read_csv('../02.Pandas/data/ㅇ주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명','행정구역명','총 인구수']]
pop.head()

,행정구역구분명,행정구역명,총 인구수
0,도,경기도,13589432
1,시군,경기도 가평군,62197
2,읍면동,경기도 가평군 가평읍,19555
3,읍면동,경기도 가평군 북면,3815
4,읍면동,경기도 가평군 상면,5669


In [23]:
# - 양주시,광주시 오포2동 데이터이상해서 수정했음 
# - 주민등록인구진계현황2로 저장함..
# 이거는 원본데이터 안바꾸고 한것

In [24]:
yang = pop[pop['행정구역명'].str.contains('경기도 양주시')]

In [25]:
yang

,행정구역구분명,행정구역명,총 인구수
379,읍면동,경기도 양주시,241346
380,읍면동,경기도 양주시 광적면,11648
381,읍면동,경기도 양주시 남면,6857
382,읍면동,경기도 양주시 백석읍,25885
383,읍면동,경기도 양주시 양주1동,5018
384,읍면동,경기도 양주시 양주2동,51121
385,읍면동,경기도 양주시 은현면,5651
386,읍면동,경기도 양주시 장흥면,11163
387,읍면동,경기도 양주시 회천1동,9545
388,읍면동,경기도 양주시 회천2동,26840


In [26]:
yang= yang[['행정구역명','총 인구수']]

In [27]:
yang.reset_index(inplace=True)

In [28]:
yang.columns

Index(['index', '행정구역명', '총 인구수'], dtype='object')

In [29]:
del yang['index']
yang

,행정구역명,총 인구수
0,경기도 양주시,241346
1,경기도 양주시 광적면,11648
2,경기도 양주시 남면,6857
3,경기도 양주시 백석읍,25885
4,경기도 양주시 양주1동,5018
5,경기도 양주시 양주2동,51121
6,경기도 양주시 은현면,5651
7,경기도 양주시 장흥면,11163
8,경기도 양주시 회천1동,9545
9,경기도 양주시 회천2동,26840


In [30]:
yang1 = yang.drop(index=0,axis=0)
yang1

,행정구역명,총 인구수
1,경기도 양주시 광적면,11648
2,경기도 양주시 남면,6857
3,경기도 양주시 백석읍,25885
4,경기도 양주시 양주1동,5018
5,경기도 양주시 양주2동,51121
6,경기도 양주시 은현면,5651
7,경기도 양주시 장흥면,11163
8,경기도 양주시 회천1동,9545
9,경기도 양주시 회천2동,26840
10,경기도 양주시 회천3동,26721


In [31]:
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
pop.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
8,시군,경기도 고양시,1075202
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738


In [32]:
pop

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
8,시군,경기도 고양시,1075202
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738
56,시군,경기도 과천시,77775
63,시군,경기도 광명시,288182
83,시군,경기도 광주시,391704
96,시군,경기도 광주시 오포2동,34293
100,시군,경기도 구리시,188876


In [33]:
except_city = ['경기도 수원시', '경기도 고양시', '경기도 용인시', '경기도 성남시', '경기도 안산시', '경기도 안양시']

pop = pop[((pop.행정구역구분명 == '시군') | (pop.행정구역구분명 == '구')) & ~(pop['행정구역명'].str.strip().isin(except_city))]
pop

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738
56,시군,경기도 과천시,77775
63,시군,경기도 광명시,288182
83,시군,경기도 광주시,391704
96,시군,경기도 광주시 오포2동,34293
100,시군,경기도 구리시,188876
109,시군,경기도 군포시,266531


In [34]:
'''
str.strip(): 문자열에서 앞뒤 공백을 제거
isin(): 주어진 리스트의 원소들 중에서 해당 데이터가 있는지 확인하고, 있으면 True를 반환
(): 파이썬에서는 ~(tilde) 연산자가 not을 의미합니다. 
따라서 ~(pop['행정구역명'].str.strip().isin(except_city))는
'pop' 데이터프레임의 '행정구역명' 열에서 'except_city' 리스트의 값들을 제외한 것을 선택하는 것을 의미
'''

"\nstr.strip(): 문자열에서 앞뒤 공백을 제거\nisin(): 주어진 리스트의 원소들 중에서 해당 데이터가 있는지 확인하고, 있으면 True를 반환\n(): 파이썬에서는 ~(tilde) 연산자가 not을 의미합니다. \n따라서 ~(pop['행정구역명'].str.strip().isin(except_city))는\n'pop' 데이터프레임의 '행정구역명' 열에서 'except_city' 리스트의 값들을 제외한 것을 선택하는 것을 의미\n"

In [35]:
pop = pop[['행정구역명', '총 인구수']]
pop.columns=['자치구','인구수']
pop.head()

,자치구,인구수
1,경기도 가평군,62197
9,경기도 고양시 덕양구,487874
31,경기도 고양시 일산동구,296590
44,경기도 고양시 일산서구,290738
56,경기도 과천시,77775


In [36]:
pop.columns

Index(['자치구', '인구수'], dtype='object')

In [37]:
gdf1.index

Index(['가평군', '고양시덕양구', '고양시일산동구', '고양시일산서구', '과천시', '광명시', '광주시', '구리시',
       '군포시', '김포시', '남양주시', '동두천시', '부천시', '성남시분당구', '성남시수정구', '성남시중원구',
       '수원시권선구', '수원시영통구', '수원시장안구', '수원시팔달구', '시흥시', '안산시단원구', '안산시상록구',
       '안성시', '안양시동안구', '안양시만안구', '양주시', '양평군', '여주시', '연천군', '오산시', '용인시기흥구',
       '용인시수지구', '용인시처인구', '의왕시', '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시',
       '화성시'],
      dtype='object', name='자치구')

In [38]:
gdf1.reset_index(inplace=True)
gdf1

,자치구,공원면적
0,가평군,511673
1,고양시덕양구,3287283
2,고양시일산동구,2722980
3,고양시일산서구,922486
4,과천시,315666
5,광명시,3397531
6,광주시,786478
7,구리시,912752
8,군포시,1455375
9,김포시,5792950


In [39]:
pop.reset_index(inplace=True)
pop

,index,자치구,인구수
0,1,경기도 가평군,62197
1,9,경기도 고양시 덕양구,487874
2,31,경기도 고양시 일산동구,296590
3,44,경기도 고양시 일산서구,290738
4,56,경기도 과천시,77775
5,63,경기도 광명시,288182
6,83,경기도 광주시,391704
7,96,경기도 광주시 오포2동,34293
8,100,경기도 구리시,188876
9,109,경기도 군포시,266531


In [40]:
pop.columns

Index(['index', '자치구', '인구수'], dtype='object')

In [41]:
del pop['index']
pop

,자치구,인구수
0,경기도 가평군,62197
1,경기도 고양시 덕양구,487874
2,경기도 고양시 일산동구,296590
3,경기도 고양시 일산서구,290738
4,경기도 과천시,77775
5,경기도 광명시,288182
6,경기도 광주시,391704
7,경기도 광주시 오포2동,34293
8,경기도 구리시,188876
9,경기도 군포시,266531


In [42]:
gdf1

,자치구,공원면적
0,가평군,511673
1,고양시덕양구,3287283
2,고양시일산동구,2722980
3,고양시일산서구,922486
4,과천시,315666
5,광명시,3397531
6,광주시,786478
7,구리시,912752
8,군포시,1455375
9,김포시,5792950


In [43]:
yang.columns = ['자치구','인구수']
yang

,자치구,인구수
0,경기도 양주시,241346
1,경기도 양주시 광적면,11648
2,경기도 양주시 남면,6857
3,경기도 양주시 백석읍,25885
4,경기도 양주시 양주1동,5018
5,경기도 양주시 양주2동,51121
6,경기도 양주시 은현면,5651
7,경기도 양주시 장흥면,11163
8,경기도 양주시 회천1동,9545
9,경기도 양주시 회천2동,26840


In [44]:
yang['자치구'] = yang.자치구.apply(lambda x: ''.join(x.split()[1:3]) if x.find('구 ') >= 0 else x.split()[1])
yang

,자치구,인구수
0,양주시,241346
1,양주시,11648
2,양주시,6857
3,양주시,25885
4,양주시,5018
5,양주시,51121
6,양주시,5651
7,양주시,11163
8,양주시,9545
9,양주시,26840


In [45]:
yang2 =yang.iloc[:1, :]
yang2

,자치구,인구수
0,양주시,241346


In [46]:
gdf1

,자치구,공원면적
0,가평군,511673
1,고양시덕양구,3287283
2,고양시일산동구,2722980
3,고양시일산서구,922486
4,과천시,315666
5,광명시,3397531
6,광주시,786478
7,구리시,912752
8,군포시,1455375
9,김포시,5792950


In [47]:
pop['자치구'] = pop.자치구.apply(lambda x: ''.join(x.split()[1:3]) if x.find('구 ') >= 0 else x.split()[1])
pop

,자치구,인구수
0,가평군,62197
1,고양시덕양구,487874
2,고양시일산동구,296590
3,고양시일산서구,290738
4,과천시,77775
5,광명시,288182
6,광주시,391704
7,광주시,34293
8,구리시,188876
9,군포시,266531


In [48]:
da2 = pd.merge(gdf1, yang2)
da2.set_index('자치구', inplace=True)
da2

,공원면적,인구수
자치구,,
양주시,2034792,241346


In [49]:
da2.reset_index(inplace=True)
da2

,자치구,공원면적,인구수
0,양주시,2034792,241346


In [50]:
da2['인당공원면적'] = (da2.인구수 / da2.공원면적 *100).round(2)
da2

,자치구,공원면적,인구수,인당공원면적
0,양주시,2034792,241346,11.86


In [51]:
da2.set_index('자치구', inplace=True)
da2

,공원면적,인구수,인당공원면적
자치구,,,
양주시,2034792,241346,11.86


In [52]:
da3 = da2[['인당공원면적']]

In [53]:
da3

,인당공원면적
자치구,
양주시,11.86


In [54]:
data5 = pd.merge(gdf1, pop)
data5.set_index('자치구', inplace=True)
data5

,공원면적,인구수
자치구,,
가평군,511673,62197
고양시덕양구,3287283,487874
고양시일산동구,2722980,296590
고양시일산서구,922486,290738
과천시,315666,77775
광명시,3397531,288182
광주시,786478,391704
광주시,786478,34293
구리시,912752,188876


In [55]:
data5['인당공원면적'] = (data5.공원면적 / data5.인구수).round(2)
data5

,공원면적,인구수,인당공원면적
자치구,,,
가평군,511673,62197,8.23
고양시덕양구,3287283,487874,6.74
고양시일산동구,2722980,296590,9.18
고양시일산서구,922486,290738,3.17
과천시,315666,77775,4.06
광명시,3397531,288182,11.79
광주시,786478,391704,2.01
광주시,786478,34293,22.93
구리시,912752,188876,4.83


In [56]:
data5 = data5[['인당공원면적']]

In [57]:
data5

,인당공원면적
자치구,
가평군,8.23
고양시덕양구,6.74
고양시일산동구,9.18
고양시일산서구,3.17
과천시,4.06
광명시,11.79
광주시,2.01
광주시,22.93
구리시,4.83


In [58]:
re = pd.concat([data5, da3], axis=0)
re

,인당공원면적
자치구,
가평군,8.23
고양시덕양구,6.74
고양시일산동구,9.18
고양시일산서구,3.17
과천시,4.06
광명시,11.79
광주시,2.01
광주시,22.93
구리시,4.83


In [59]:
type(data5.인당공원면적[0])

numpy.float64

In [60]:
map = folium.Map([37.55, 126.98], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data, # 맵데이터
    data=re.인당공원면적, # 단계구분도로 보여줄 데이터
    columns=[re.index, re.인당공원면적], # dataframe에서 추출할 컬럼(항목)
    fill_color='YlGnBu',   # Colormap
    key_on='properties.name' # 지도에서 join할 항목
).add_to(map)
for gu_name in re.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
                              html=f'<div style="font-size: 10pt">{gu_name}</div>')
    ).add_to(map)
map
title_html = '<h3 align="center" style="font-size:20px">서울시 자치구별 인당공원면적</h3>' # 지도 제목달기
map.get_root().html.add_child(folium.Element(title_html))
map